In [2]:
import sys
import os
sys.path.append('/home/azhar04/project/1. dev/quantum-dev/ShotOptimized-ADAPT-VQE/')
# sys.path.append('/home/alfarialstudio/ShotOptimization-ADAPT-VQE/')

import json
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from openfermion import MolecularData
from openfermion.transforms import jordan_wigner, get_fermion_operator, freeze_orbitals
from openfermionpyscf import run_pyscf

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorEstimator, StatevectorSampler
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import Pauli
from qiskit.circuit import ParameterVector

from src.pools import QE
from src.molecules import create_h2, create_h3, create_lih
from src.utilities import to_qiskit_operator, get_eigenvalues, get_probability_distribution

from qiskit import QuantumCircuit

# PARAMETERS
R = 0.86
SHOTS = 1024
N_EXP = 50
N_0 = 10
SEED = None
PLOT = True
XLIM = 0.1

PauliX = Pauli("X")
PauliZ = Pauli("Z")
PauliI = Pauli("I")
PauliY = Pauli("Y")

# Molecule Type
# molecule = create_h2(R)
# molecule = create_h3(R)
molecule = create_lih(R)

In [3]:
molecule.__dict__

{'geometry': [['Li', [0, 0, 0]], ['H', [0, 0, 0.86]]],
 'basis': 'sto-3g',
 'multiplicity': 1,
 'charge': 0,
 'description': 'LiH',
 'name': 'H1-Li1_sto-3g_singlet_LiH',
 'filename': '/home/azhar04/anaconda3/envs/qiskit-1-2/lib/python3.12/site-packages/openfermion/testing/data/H1-Li1_sto-3g_singlet_LiH',
 'n_atoms': 2,
 'atoms': ['H', 'Li'],
 'protons': [1, 3],
 'n_electrons': 4,
 'n_orbitals': 6,
 'n_qubits': 12,
 'nuclear_repulsion': 1.8459670148372096,
 'hf_energy': -7.673720441434493,
 'orbital_energies': array([-2.41119173, -0.31166354,  0.07164136,  0.15283446,  0.15283446,
         0.59696111]),
 'mp2_energy': None,
 'cisd_energy': None,
 'fci_energy': -7.69167167746149,
 'ccsd_energy': -7.691665848069231,
 'general_calculations': {},
 '_canonical_orbitals': array([[ 9.78756371e-01, -2.97420246e-01, -1.77726688e-01,
         -1.50785926e-18, -1.17395773e-17, -2.49245305e-01],
        [ 8.33579043e-03,  3.77375236e-01,  8.55779005e-01,
          7.53844296e-17, -2.71688276e-16, -

In [1]:
from openfermion.ops import QubitOperator

In [2]:
hamiltonian = 0.5 * QubitOperator('X0 X5') + 0.3 * QubitOperator('Z0')

In [3]:
hamiltonian

0.5 [X0 X5] +
0.3 [Z0]

In [11]:
a = hamiltonian - hamiltonian
print(a)
print(type(a))
print(a==0)
print(a==None)
print(hamiltonian.induced_norm())
print(a.induced_norm()==0)

0
<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>
False
False
0.8
True


In [6]:

# Hamiltonian
fermionic_hamiltonian = molecule.get_molecular_hamiltonian()

print("Number of LiH Orbitals:", molecule.n_orbitals)

qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)

qiskit_hamiltonian = to_qiskit_operator(qubit_hamiltonian)

print("Number of LiH Qubits (Full):", qiskit_hamiltonian.num_qubits)


frozen_orbitals = [0,1]
freezed_hamiltonian = get_fermion_operator(fermionic_hamiltonian)
# print("Get Fermion Operator", hamiltonian)
freezed_hamiltonian = freeze_orbitals(freezed_hamiltonian, frozen_orbitals)
# print("Freeze Orbitals", hamiltonian)
qubit_hamiltonian_freezed = jordan_wigner(freezed_hamiltonian)
qiskit_hamiltonian_freezed = to_qiskit_operator(qubit_hamiltonian_freezed)

print("Number of LiH Qubits (Freezed):", qiskit_hamiltonian_freezed.num_qubits)


Number of LiH Orbitals: 6
Number of LiH Qubits (Full): 12
Number of LiH Qubits (Freezed): 10


In [16]:
len(qiskit_hamiltonian.group_commuting(qubit_wise=True))

154

In [17]:
len(qiskit_hamiltonian_freezed.group_commuting(qubit_wise=True))

64

In [8]:
len(qiskit_hamiltonian)

631

In [9]:
len(qiskit_hamiltonian_freezed)

276

In [26]:
# Hardware Efficient Circuit
ansatz = EfficientSU2(qiskit_hamiltonian.num_qubits, reps=0)

parameters_value = np.zeros(ansatz.num_parameters)

# ---

# print(ansatz)

print("Number of Parameters:", ansatz.num_parameters)

# Qiskit Estimator
estimator = StatevectorEstimator()
pub = (ansatz, qiskit_hamiltonian, parameters_value)
job = estimator.run([pub])
exp_vals_estimator = job.result()[0].data.evs
print(exp_vals_estimator)

Number of Parameters: 24
1.845967014837202


In [27]:
# Hardware Efficient Circuit
ansatz = EfficientSU2(qiskit_hamiltonian_freezed.num_qubits, reps=0)

parameters_value = np.zeros(ansatz.num_parameters)

# ---

# print(ansatz)

print("Number of Parameters:", ansatz.num_parameters)

# Qiskit Estimator
estimator = StatevectorEstimator()
pub = (ansatz, qiskit_hamiltonian_freezed, parameters_value)
job = estimator.run([pub])
exp_vals_estimator = job.result()[0].data.evs
print(exp_vals_estimator)

Number of Parameters: 20
-6.528399107522976
